In [ ]:
# !git clone https://github.com/Changyoon-Lee/big_contest.git

Cloning into 'big_contest'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 360 (delta 0), reused 3 (delta 0), pack-reused 356
Receiving objects: 100% (360/360), 130.76 MiB | 24.60 MiB/s, done.
Resolving deltas: 100% (167/167), done.
Checking out files: 100% (230/230), done.


In [130]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [140]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras import datasets, layers, models
from sklearn.model_selection import train_test_split
import seaborn as sns
from scipy import stats

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from tqdm.notebook import tqdm

In [133]:
with open('/content/drive/My Drive/빅콘/merged_dataframe.pickle', 'rb') as f:
    data = pickle.load(f)

In [134]:
data.columns

Index(['역명', '역명_label', 'date', '시간', '시간_label', '하차', '승차', '요일',
       '요일_label', '공휴일', '주말제외공휴일', 'corona_stage', '행정구역_확진자', '전국_확진자',
       '검색트랜드', '평균하차수'],
      dtype='object')

In [ ]:
numeric_pipeline = Pipeline([
                             ('scaler', StandardScaler()
                             )])
cgr_pipeline = Pipeline([
                         ('onehot', OneHotEncoder())
                         ])
df_pipeline = ColumnTransformer([
                                 ('num_pipeline', numeric_pipeline, numeric_columns),
                                 ('cgr_pipeline', cgr_pipeline, cgr_columns)
])

In [139]:
score_train, score_test = [],[]

In [142]:
for i in tqdm(range(175)):
  prac = data[data['역명_label']==i]

  numeric_columns = ['승차',
          '공휴일', '주말제외공휴일', 'corona_stage', '행정구역_확진자', '전국_확진자',
        '검색트랜드', '평균하차수']
  cgr_columns = ['요일_label']
  prac = prac[['승차','공휴일', '주말제외공휴일', 'corona_stage', '행정구역_확진자', '전국_확진자',
        '검색트랜드', '평균하차수','요일_label','하차']]

  y = prac['하차']
  X = prac.drop('하차',axis=1)
  X_train, X_test, y_train, y_test = train_test_split(X, y,shuffle=False)
  X_train.shape, X_test.shape, y_train.shape, y_test.shape
  X_train_ready = df_pipeline.fit_transform(X_train)
  lin_reg = LinearRegression()
  lin_reg.fit(X_train_ready, y_train)
  score_train.append(lin_reg.score(X_train_ready, y_train))
  X_test_ready = df_pipeline.fit_transform(X_test)
  score_test.append(lin_reg.score(X_test_ready, y_test))

In [147]:
# y에 대한 분포
# sns.distplot(prac['하차'], rug=True, rug_kws={'alpha': 0.05})
# stats.probplot(prac['하차'], plot=plt)

In [145]:
np.mean(score_train),np.mean(score_test)

(0.9279874903620878, 0.8724320758653994)